In [1]:
# Web Scraping
!pip install BeautifulSoup4
!pip install requests
!pip install lxml

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from lxml import html
import lxml.etree

print('ALL libraries imported!')

ALL libraries imported!


In [4]:
# get html from wiki, create Soup object
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").content
soup = BeautifulSoup(source, "html.parser")
print('soup object created')

soup object created


In [5]:
# iterate wikitable to get data from HTML, store to list
data = []
table = soup.find(class_='wikitable')
rows = table.find_all('tr')

for row in rows:
    data.append([td.text.strip() for td in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df.head()

,PostalCode,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [6]:
# clean data
# clean Borough = Not Assigned or Neightbourhood is blank
df['Neighbourhood'].replace('', np.nan, inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,NaN
2,M2A,Not assigned,NaN
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [7]:
# Remove rows with NaN
df.dropna(subset=['Neighbourhood'], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(20)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [8]:
#dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PostalCode     103 non-null    object
 1   Borough        103 non-null    object
 2   Neighbourhood  103 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


In [9]:
print("shape ",df.shape)

shape  (103, 3)


In [10]:
# reset index to PostalCode for join table later
df = df.set_index('PostalCode')
df.head()

,Borough,Neighbourhood
PostalCode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
# Add Geospatial Data

#Get data lat/long data from csv
coor_df = pd.read_csv('http://cocl.us/Geospatial_data')

#rename columns and set the index to be Postcode
coor_df.columns = ["PostalCode", "Latitude", "Longitude"]
if(coor_df.index.name != 'PostalCode'):
    coor_df = coor_df.set_index('PostalCode')
    
coor_df

,Latitude,Longitude
PostalCode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
...,...,...
M9N,43.706876,-79.518188
M9P,43.696319,-79.532242
M9R,43.688905,-79.554724


In [12]:
df = df.join(coor_df).reset_index()
df

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
